In [2]:
from ultralytics import YOLO
import cv2
from collections import OrderedDict
import numpy as np
import yaml
import os

os.environ['CLEARML_CONFIG_FILE'] = "clearml.conf"

from clearml import Task, InputModel

with open('config.yaml','r') as f:
    configModel = yaml.safe_load(f)

task = Task.init(project_name='OCR Model-eFishery', task_name='OCR Detection and Text Extraction', task_type="inference")
inputModel = InputModel(project="OCR Model-eFishery", name=configModel["model-config"]["YOLO-model"])
task.connect(inputModel)
pathToModel = inputModel.get_local_copy()

model = YOLO(pathToModel)
imgDelimiter = cv2.imread(configModel["model-config"]["delimiter"])
gcv_api_key_path = configModel["model-config"]["vision-key"]
imgSize = configModel["model-config"]["image-size"]

ClearML Task: overwriting (reusing) task id=f84a1314c87946849abc6006a7235837
2023-05-11 08:20:08,951 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2023-05-11 08:20:08,982 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/8e9052522ad44137a215e9722d9ce3d8/experiments/f84a1314c87946849abc6006a7235837/output/log
2023-05-11 08:20:30,207 - clearml.storage - INFO - Downloading: 5.00MB / 83.58MB @ 1.11MBs from https://files.clear.ml/OCR%20Model-eFishery/YOLOv8%20Training%20Model%20Vibrio.3c152fda4726421b9d8950ec1627c97a/models/best.pt
2023-05-11 08:20:32,841 - clearml.storage - INFO - Downloading: 10.00MB / 83.58MB @ 1.90MBs from https://files.clear.ml/OCR%20Model-eFishery/YOLOv8%20Training%20Model%20Vibrio.3c152fda4726421b9d8950ec1627c97a/models/best.pt
2023-05-11 08:20:34,572 - clearml.storage - INFO - Downloading: 15.00MB / 83.58MB @ 2.89MBs from https://files.clear.

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [3]:
img = cv2.imread("vibrio/vibrio_mae/3-52-vibrio.jpg")

print(img.shape)

scalingH, scalingW = img.shape[0]/imgSize, img.shape[1]/imgSize
data = cv2.resize(img, (imgSize, imgSize))

results = model.predict(data, imgsz = imgSize,
                        conf = configModel["model-config"]["conf"], iou = configModel["model-config"]["iou"],
                        save = configModel["model-config"]["save-mode"], save_conf = configModel["model-config"]["save-mode"],
                        save_crop = configModel["model-config"]["save-mode"], save_txt = configModel["model-config"]["save-mode"],
                        device = configModel["model-config"]["device-mode"])

(3000, 4000, 3)



0: 640x640 22 greens, 30 yellows, 973.7ms
Speed: 6.0ms preprocess, 973.7ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict
1 label saved to runs\detect\predict\labels


In [17]:
classes = results[0].names
print(classes)

{0: 'black', 1: 'green', 2: 'yellow'}


In [14]:
totalFound = {}
for boxes in results[0].boxes:
    for box in boxes:
        if int(box.cls) in totalFound.keys():
            totalFound[int(box.cls)] = totalFound[int(box.cls)] + 1
        else:
            totalFound[int(box.cls)] = 1

In [16]:
print(totalFound)

{2: 30, 1: 22}


In [20]:
finalDict = {}
dictDataEntity = {}
orderedDict = OrderedDict(sorted(totalFound.items()))
for key, value in orderedDict.items():
    for classKey, classValue in classes.items(): 
        if key == classKey:
             finalDict[classValue] = value

print(finalDict)

{'green': 22, 'yellow': 30}


In [23]:
for label in classes.values():
        if label not in finalDict.keys():
            finalDict[label] = "0"

In [24]:
finalDict

{'green': 22, 'yellow': 30, 'black': '0'}

2023-05-11 09:37:32,100 - clearml.Task - WARNING - Task f84a1314c87946849abc6006a7235837 was reset! if state is consistent we shall terminate.
